In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd
import pickle
from fp.fp import FreeProxy

In [2]:
with open("goods.pickle", "rb") as g:
    goods = pickle.load(g) #불러오기

In [3]:
def current_review():
    for k in range(0,10):
        item = driver.find_element(By.CLASS_NAME, 'prd_name').text
        items.append(item) #상품명
        time.sleep(1)
        try:
            user_id = driver.find_elements(By.CLASS_NAME, 'id')[k]
            onclick = user_id.get_attribute('onclick')
            id_key = onclick.split("'")[-2]
        except IndexError:
            id_key = np.nan
        user.append(id_key) #사용자 id

        try: #피부 타입은 없는 경우도 있다
            t = driver.find_elements(By.CLASS_NAME, 'tag')[k].text
        except IndexError:
            t = np.nan
        type.append(t) #피부 타입

        driver.implicitly_wait(5)
        r = driver.find_elements(By.CLASS_NAME, 'point')[1:] #총 평점 제외
        r = r[k].text
        rate.append(r) #평점
        t = driver.find_elements(By.CLASS_NAME, 'txt_inner')[k].text
        txt.append(t) #리뷰
        d = driver.find_elements(By.CLASS_NAME, 'date')[k].text
        date.append(d) #날짜

        try: #한달이상사용 및 재구매 여부는 없는 경우도 있다
            m = driver.find_elements(By.CLASS_NAME, 'point_flag')[k].text
        except IndexError:
            m = np.nan
        month_rebuy.append(m) #한달이상 사용 및 재구매 여부

In [8]:
# selenium header에 알맞은 형태로 변환
proxy_server = proxy.split('//')[1]


'189.232.96.231:8080'

In [7]:
# free-proxy get
proxy = FreeProxy(rand = True, https = True).get()

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

# selenium header에 알맞은 형태로 변환
proxy_server = proxy.split('//')[1]
chrome_options.add_argument(f'--proxy-server={proxy_server}')
service = Service(executable_path=ChromeDriverManager().install())

In [ ]:
#반복문으로 구현
items = list(); user = list(); type = list(); rate = list(); txt = list(); date = list()
month_rebuy = list()
driver = webdriver.Chrome(service=service, options=chrome_options)

### 긁어오지 않은 부분부터 시행해야
for _,j in enumerate(goods.loc[105:114, 'url']):
    resp = requests.get(j)
    if resp.status_code == requests.codes.ok: #status_code == 200인 경우만
        driver.get(j) #개별 상품 페이지로 이동
        time.sleep(0.5)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤 끝까지 내리기
        try:
            review = driver.find_element(By.CSS_SELECTOR, '#reviewInfo')
            review.click() #리뷰 클릭
            driver.implicitly_wait(10)
            temp = driver.find_element(By.CSS_SELECTOR, "#gdasSort > li.is-layer.on")
            temp.click() #리뷰 유용한 순 클릭
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤 끝까지 내리기
            
            try: #리뷰가 있는 경우
                current_review() #첫페이지 리뷰 정보 가져오기
                for i in range(2,11): #페이지 이동 : 10페이지까지
                    next = driver.find_element(By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a:nth-child({})".format(i))
                    next.send_keys(Keys.ENTER)
                    driver.implicitly_wait(3)
                    current_review()
            except NoSuchElementException: #리뷰 없는 경우 : 리뷰가 100개 이하인 경우도 같은 메시지
                print(_, "리뷰가 없습니다")
        except NoSuchElementException: #페이지 로딩 안 되는 경우
            print(_, "페이지 로딩 실패")
        except: #기타 에러 발생한 경우 반복문 중지
            print("ERROR: ", _)
            driver.quit()
            break

In [71]:
# 리스트 길이 동일한지 확인
## 리뷰가 제대로 크롤링되지 않은 경우 items, user, type의 길이가 더 길 수 있다: list[:-1]로 잘라주어야
print(len(items), len(user), len(type), len(rate), len(txt), len(date), len(month_rebuy))

181 181 180 180 180 180 180


In [72]:
# 리스트 > 데이터프레임으로 변경
## 재구매 및 한달사용 여부의 경우 별도 데이터프레임으로 저장
df = pd.DataFrame({"상품명2":items[:-1], "리뷰자":user[:-1], "피부타입":type, "평점":rate, "리뷰":txt, "날짜":date})
temp = pd.DataFrame({"상품명2":items[:-1], "재구매및한달":month_rebuy})
temp2 = temp.groupby('상품명2')['재구매및한달'].value_counts()
temp2 = temp2.to_frame()
df.tail(1) #마지막으로 크롤링한 리뷰 확인하기

,상품명2,리뷰자,피부타입,평점,리뷰,날짜
179,[단독기획] 스킨푸드 캐롯 카로틴 수딩 토너 300ml+200ml,WVhJUWFHTFFhY3NwSUxEcHRPM3FTZz09,NaN,5점만점에 5점,당근 패드 속에 있는 토너느낌이랑 비슷해요 냄새도요!! 수분감이 엄청 많고 진정되는...,2023.04.23


In [73]:
with open("review.pickle", "rb") as g:
    previous = pickle.load(g) #이전 크롤링 결과 불러오기

with open("month_rebuy.pickle", "rb") as f:
    previous_month = pickle.load(f) # 이전 재구매 여부 불러오기

In [74]:
# concat : 크롤링 결과
new_df = pd.concat([previous, df])

# 중복 제거 : 모든 컬럼 기준
# 댓글 페이지가 제대로 로딩되지 않은 경우 중복건이 발생하는 것으로 보임
new_df.drop_duplicates(inplace=True)
new_df.shape

(5813, 6)

In [75]:
new_df['상품명2'].nunique() # 몇 개 상품의 리뷰를 가져왔나

94

In [76]:
temp2 = temp2.rename(columns={"재구매및한달":"count"}).reset_index(drop=False)

In [77]:
# concat : 재구매 여부
new_temp = pd.concat([previous_month, temp2])
new_temp.drop_duplicates(inplace=True) #중복 제거
new_temp.shape

(223, 3)

In [78]:
with open("review.pickle", "wb") as g:
    pickle.dump(new_df, g) #리뷰 크롤링 결과 저장

In [79]:
with open("month_rebuy.pickle", "wb") as f:
    pickle.dump(new_temp, f) # 상품별 재구매 및 한달사용 여부 저장